In [ ]:
import math 
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite
import networkx as nx
from ortools.linear_solver import pywraplp as OR
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
import copy
import pickle
from bokeh import palettes
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import (GraphRenderer, Circle, MultiLine, StaticLayoutProvider,
                          HoverTool, TapTool, EdgesAndLinkedNodes, NodesAndLinkedEdges,
                          ColumnDataSource, LabelSet, NodesOnly)
from bipartite_matching import *
output_notebook()

# Taxi Routing Bipartite Matching Lab

**Objectives:**

* Construct the taxi-routing problem as a bipartite graph.
* Use the maximum cardinality matching problem to solve the taxi-routing problem.
* Compare the optimal solution to the taxi-routing problem with the original routing.
    
<font color='red'> **Instructor Comments** </font>

<font color='blue'> **Solutions** </font>

## Part 1: Bipartite Graph Formulation 

Load the taxi trips information as well as NYC street nodes and arcs.

In [ ]:
trips_df = pd.read_csv('data/2013-09-01_trip_data_manhattan.csv').drop(columns='id')
nodes_df = pd.read_csv('data/nyc_nodes_manhattan.csv').drop(columns='Unnamed: 0')
arcs_df = pd.read_csv('data/nyc_links_manhattan.csv').drop(columns='Unnamed: 0')
times_df = pd.read_csv('data/times.csv', index_col =0)
times_df.columns = times_df.columns.astype(float)

Look at an example input. Each trip consists of the unique vehicle id (medallion), location id of the pickup node and drop-off node, pickup time and drop-off time.

In [ ]:
# A list of example trip_ids
ex_trips = [68326, 69501, 70802, 68619, 69802, 70142, 68751, 69558, 70296, 68272]
# Locate the corresponding trip information
trips = trips_df.iloc[ex_trips]
trips

Consider a bipartite graph $G = (D, P, E)$ where $D$ and $P$ are disjoint sets corresponding to the "left" and "right" sides of the graph, and $E$ is the set of edges, each of which has exactly one endpoint in $D$ and one endpoint in $P$.

Let $D = \{(d_1, T^d_{1}),(d_2, T^d_{2}),\ldots,(d_n, T^d_{n})\}$ denote the set of drop-off nodes, $P = \{(p_1, T^p_{1}),(p_2, T^p_{2}),\ldots,(p_n, T^p_{n})\}$ denote the set of pickup nodes, where there are $n$ taxi trips to be covered. We can express the set of taxi trips $N = \{(p_i, T^p_{i}, d_i, T^d_{i}): i\in \{1,\ldots,n\}\}$; that is, the taxi history shows that the $i$th trip picks up a passenger at time $T^p_{i}$ at point $p_i$ and drops off the fare at time $T^d_{i}$ at point $d_i$.

Construct nodes and edges for the bipartite graph.

In [ ]:
# Intialize nodes and edges
DO_nodes = list()
PU_nodes = list()
edges = list()
# Initialize a dict that maps a PU node to a DO node
PUtoDO = dict()

Each node is a tuple of (location_id, time, trip_id, "DO"/"PU")

In [ ]:
for index, row in trips.iterrows():
    s = row['start_node']
    t = row['end_node']
    s_t = row['start_time']
    t_t = s_t + row['trip_time']
    DO_node = (int(t), t_t, index, 'DO')
    PU_node = (int(s), s_t, index, 'PU')
    DO_nodes.append(DO_node)
    PU_nodes.append(PU_node)
    PUtoDO[PU_node] = DO_node

**Q:** Assume that a taxi handles two consecutive trips $i$ and $j$ (in order), where the drop-off time at location $d_i$ is $T^d_i$ and the pickup time at location $p_j$ is $T^p_j$. What is the *elapsed time* between the drop-off time of trip $i$ and the pickup time of trip $j$ (express in terms of $T^d_i$ and $T^p_j$) ?

**A:** <font color='blue'> The *elapsed time* is $T^p_j - T^d_i$.</font>

**Q:** Assume that $time(d_i, p_j)$ is the time needed to travel between location point $d_i$ and location point $p_j$. What relationship must hold between the *elasped time* between the drop-off time of trip $i$ and the pickup time of trip $j$ and $time(d_i, p_j)$ in order for a taxi to cover both trip $i$ and trip $j$ (express in terms of $time(d_i, p_j), T^d_i$ and $T^p_j$)? What does this inequality mean?

**A:** <font color='blue'> $time(d_i, p_j) \leq T^p_j - T^d_i$. It means that a taxi can reach the new pickup at $p_j$ in time after dropping off the fare at $d_i$. </font>

**Q:** In the previous question, what if the taxi can only stay for a maximum of $\delta$ minutes? What is the new inequality and what does it mean?

**A:** <font color='blue'> $T^p_j - T^d_i - \delta \leq time(d_i, p_j) \leq T^p_j - T^d_i$. The new inequality means that a taxi can reach the new pickup at $p_j$ in time after dropping off the fare at $d_i$, and also doesn’t need to wait more than $\delta$ minutes for the new pickup to be ready. </font>

**Q:** Based on the new inequality you just estabilished, can you define the set $E$ mathematically?

**A:** <font color='blue'> $$E = \{\{(d_i, T^d_{i}),(p_j, T^p_{j})\}: T^p_{j} - T^d_{i}-\delta \leq time(d_i, p_j) \leq T^p_{j} - T^d_{i}\}$$ </font>

In [ ]:
# Sort the nodes by time
DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
PU_nodes = sorted(PU_nodes, key = lambda x: x[1])

# Specify the edges
max_waiting_time = 10 # delta

for DO_node in DO_nodes:
    for PU_node in PU_nodes:
        if PU_node[1] > DO_node[1] + max_waiting_time:
            break
        else:
            if PU_node[1] >= DO_node[1]:
                time = times_df.at[(DO_node[0], PU_node[0])]
                if ((PU_node[1] - DO_node[1]) - max_waiting_time  <= time) & (time <= (PU_node[1] - DO_node[1])):
                    edges.append((DO_node, PU_node))

Generate the bipartite graph

In [ ]:
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(DO_nodes, bipartite=0)
B.add_nodes_from(PU_nodes, bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_edges_from(edges)

**Theorem**

If a maximum cardinality matching is of size $m$, then the minimum number of taxis needed to cover all $n$ trips is $n-m$.

In [ ]:
match = nx.bipartite.maximum_matching(B, DO_nodes)
print('Size of max cardinality matching:', int(len(match)/2)) # divided by two because the output edges are directed

**Q:** What is the minimum number of taxis needed to cover all the trips according to the size of the maximum cardinality matching?

**A:** <font color='blue'> 4 taxis.</font>

We can use the information given by maximum cardinality matching to track down the optimal taxi trajectories. 

In [ ]:
opt_paths = match_to_path(match, trips)

Plot the bipartite graph

In [ ]:
plot_ex_bipartite(B, match, opt_paths, True)

**Q:** Interpret one optimal taxi trajectory that contains three trips based on the graph above using the location ids (remember each node is a tuple of (location_id, time, trip_id, "DO"/"PU")).

**A:** <font color='blue'> 1447->1762->1163->24->79->931 (or 2119->275->1494->1809->20->1323)</font>

Plot the corresponding taxi paths on the map

In [ ]:
G = street_network(nodes_df, arcs_df, weight = 'trip_time')
plot_taxi_route(G, opt_paths, nodes_df)

## Part 2: Bipartite Graph Formulation (At Scale)

Now, instead of selecting a sample selection of taxi trips, try filter the trips by time window of interest. The following example selects all the trips from 5 pm to 5:15 pm.

In [ ]:
# Filter trips by time window of interest
start_time = 1020
end_time = 1035
trips = trips_df.copy()
trips = trips[(trips.start_time >= start_time) & 
              (trips.start_time + trips.trip_time <= end_time)].copy()
trips.start_time = trips.start_time - start_time

Nodes and edges are defined similarly.

In [ ]:
# Intialize nodes and edges
DO_nodes = list()
PU_nodes = list()
edges = list()
# Initialize a dict that maps a PU node to a DO node
PUtoDO = dict()

In [ ]:
# Specify nodes - each node is a tuple of (location_id, time, trip_id, "DO"/"PU")
for index, row in trips.iterrows():
    s = row['start_node']
    t = row['end_node']
    s_t = row['start_time']
    t_t = s_t + row['trip_time']
    DO_node = (int(t), t_t, index, 'DO')
    PU_node = (int(s), s_t, index, 'PU')
    DO_nodes.append(DO_node)
    PU_nodes.append(PU_node)
    PUtoDO[PU_node] = DO_node

In [ ]:
# Sort the nodes by time
DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
PU_nodes = sorted(PU_nodes, key = lambda x: x[1])

# Specify edges
max_waiting_time = 10

for DO_node in DO_nodes:
    for PU_node in PU_nodes:
        if PU_node[1] > DO_node[1] + max_waiting_time:
            break
        else:
            if PU_node[1] >= DO_node[1]:
                time = times_df.at[(DO_node[0], PU_node[0])]
                if ((PU_node[1] - DO_node[1]) - max_waiting_time  <= time) & (time <= (PU_node[1] - DO_node[1])):
                    edges.append((DO_node, PU_node))

In [ ]:
# load the model
B = nx.Graph()
# Add nodes with the node attribute "bipartite"
B.add_nodes_from(DO_nodes, bipartite=0)
B.add_nodes_from(PU_nodes, bipartite=1)
# Add edges only between nodes of opposite node sets
B.add_edges_from(edges)

In [ ]:
match = nx.bipartite.maximum_matching(B, DO_nodes)

In [ ]:
print('size of max cardinality matching:', len(match) / 2)

In [ ]:
# Calculate number of unmatched nodes (minimum number of taxis needed)
# TODO: Set num_taxi equal to the number of unmatched nodes in terms of the number of drop-off nodes and the size of the matching
# num_taxi = XXX

### BEGIN SOLUTION
num_taxi = len(DO_nodes) - len(match)/2
### END SOLUTION

print('min number of taxis needed: ', num_taxi)

We will write the bipartite matching problem in a format that we will use extensively as this course proceeds, which we will generally call to be Integer Programming formulations.

Restating the bipartite matching problem, we are given a bipartite graph $G=(N,E)$ (which is undirected) with a node set $N= P \cup D$ (for the pickups and drop-offs), where each edge $\{i,j\} \in E$ has one endpoint $i \in P$ and the other endpoint $j \in D$. We wish to select a subset of edges $M \subseteq E$ with the property that each node in $N$ is an endpoint of at most one edge in $M$; such a subset of edges is called a matching. The bipartite matching problem is, given such an input, to find the matching with maximum number of edges.

We can think about writing down a matching $M$ in the following way – for each edge $e \in E$, we indicate "yes" or "no", whether or not to include $e$ in $M$. We can specify this answer by having a so-called "decision variable" $x(e)$ for each edge $e \in E$, and then indicating whether that variable should be set to 1 (meaning "yes") or to 0 (meaning "no"). 

But given a sequence of "yes"s and "no"s, how do we know that the selected set of edges actually does correspond to a matching? We need to check that for each node, we have selected at most one edge with that endpoint. We introduce a bit of notation to help: let 
$$ \delta (v) = \{ e = \{v,w\} : \{v,w\} \in E \}; $$
that is, $\delta (v)$ is the set of *all* edges in $E$ that have node $v$ as one of its endpoints. For $M$ to be a matching, we need to select at most one edge in $\delta (v)$ to be in $M$, *for each* $v \in N = P \cup D$. 

If we think about how we used the values 0 and 1 for each edge $e \in E$, to set $x(e)$, that means that, for each $v \in N = P \cup D$, we have at most one $x(e )$  set to 1, and the rest are set to 0. Equivalently, we can say that we must have the sum of these variables be at most 1; that is $\sum_{e \in \delta (v)} x(e ) \leq 1$.
Our goal is to find a matching that has as many edges as possible, and so we want $\sum_{e \in E} x (e )$ to be as big as possible. So this means that we can think of the bipartite matching problem as the following concisely written optimization problem:


$$\begin{align*}
\max \quad & \sum_{e \in E}x_e \\
\text{s.t.} \quad &  \sum_{e \in \delta(v)} x_e \leq 1 \quad \forall v \in P \cup D & (1)\\
\quad & x_e \in \{0,1\} \quad \forall e \in E & (2)\\
\end{align*}$$

In [ ]:
# A dictionary that stores the list of edges adjacent to node
incident = dict()
for v in (DO_nodes + PU_nodes):
    incident[v] = [edge for edge in edges if v in edge]

In [ ]:
solver = OR.Solver('taxi_bipartite', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
# Decision variables
x = {}  
for e in B.edges:
    x[e] = solver.IntVar(0, 1, ('(%s)' % str(e)))

solver.Maximize(sum(x[e] for e in edges))

for v in (DO_nodes + PU_nodes):
    if v in incident.keys():
        solver.Add(sum(x[e] for e in incident[v]) <= 1)

In [ ]:
solver.Solve()
print('Solution:')
print('Objective value =', solver.Objective().Value())

We can also check our answer by reducing the max matching problem to a max flow problem - the max flow value should be equal to the size of max cardinality matching.

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(PU_nodes + DO_nodes + ['s'] + ['t'])
for edge in edges:
    G.add_edge(edge[0], edge[1], capacity = 1)
for DO_node in DO_nodes:
    G.add_edge('s', DO_node, capacity = 1)
for PU_node in PU_nodes:
    G.add_edge(PU_node, 't', capacity = 1)

In [ ]:
flow_value, flow_dict = nx.maximum_flow(G, "s", "t")

In [ ]:
print('max flow value: ', flow_value)

In [ ]:
# Try another alogrithm to solve max flow
from networkx.algorithms.flow import shortest_augmenting_path
print('max flow value: ', nx.maximum_flow(G, "s", "t", flow_func=shortest_augmenting_path)[0])

We can turn the process of building a bipartite graph and solving the max cardinality matching into a function

In [ ]:
def max_match(start_time, end_time, trips_df, max_waiting_time):
    # filter trips by time window of interest
    trips = trips_df.copy()
    trips = trips[(trips.start_time >= start_time) & 
                  (trips.start_time + trips.trip_time <= end_time)].copy()
    trips.start_time = trips.start_time - start_time

    # create a dictionary to map the location pairs to trip time
    loc_time = dict()
    for index, row in arcs_df.iterrows():
        i = row['start']
        j = row['end']
        delay = row['trip_time']
        loc_time[(i, j)] = delay
    # Intialize nodes and edges
    DO_nodes = list()
    PU_nodes = list()
    edges = list()
    # Initialize a dict that maps a PU node to a DO node
    PUtoDO = dict()
    # Specify nodes - each node is a tuple of (location_id, time, trip_id, "DO"/"PU")
    for index, row in trips.iterrows():
        s = row['start_node']
        t = row['end_node']
        s_t = row['start_time']
        t_t = s_t + row['trip_time']
        DO_node = (int(t), t_t, index, 'DO')
        PU_node = (int(s), s_t, index, 'PU')
        DO_nodes.append(DO_node)
        PU_nodes.append(PU_node)
        PUtoDO[PU_node] = DO_node
    DO_nodes = sorted(DO_nodes, key = lambda x: x[1])
    PU_nodes = sorted(PU_nodes, key = lambda x: x[1])
    
    # Specify edges
    for DO_node in DO_nodes:
        for PU_node in PU_nodes:
            if PU_node[1] > DO_node[1] + max_waiting_time:
                break
            else:
                if PU_node[1] >= DO_node[1]:
                    time = times_df.at[(DO_node[0], PU_node[0])]
                    if ((PU_node[1] - DO_node[1]) - max_waiting_time <= time) & (time <= (PU_node[1] - DO_node[1])):
                        edges.append((DO_node, PU_node))
    
    # load the model
    B = nx.Graph()
    # Add nodes with the node attribute "bipartite"
    B.add_nodes_from(DO_nodes, bipartite=0)
    B.add_nodes_from(PU_nodes, bipartite=1)
    # Add edges only between nodes of opposite node sets
    B.add_edges_from(edges)
    
    top_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}
    match = nx.bipartite.maximum_matching(B, top_nodes)
    num_taxi = len(DO_nodes) - len(match)/2

    return B, match, num_taxi, trips

In [ ]:
fifteen_min_B, fifteen_min_match, fifteen_min_vnum_taxi, fifteen_min_trips = max_match(1020, 1035, trips_df, 10)
print('max cardinality matching:', len(fifteen_min_match)/2)
print('min number of taxis needed to cover all trips:', fifteen_min_vnum_taxi)

Try expanding the time window to 5-6 pm and restricting the max waiting time to 5 mininutes.

In [ ]:
# TODO: Uncomment the code below and change START_TIME, END_TIME, and MAX_WAIT_TIME.
# one_hr_B, one_hr_match, one_hr_num_taxi, one_hr_trips =  max_match(START_TIME, END_TIME, trips_df, MAX_WAIT_TIME)

### BEGIN SOLUTION
one_hr_B, one_hr_match, one_hr_num_taxi, one_hr_trips = max_match(1020, 1080, trips_df, 5)
### END SOLUTION
print('max cardinality matching:', len(one_hr_match)/2)
print('min number of taxis needed to cover all trips:', one_hr_num_taxi)

## Part 3: Compare one day's taxi routing solutions

In [ ]:
# Load the result of a day's data with a max waiting time of 10 min
with open('data/day_match.pkl', 'rb') as f:
    match = pickle.load(f)

In [ ]:
# Get the optimal taxi paths
opt_paths = match_to_path(match, trips_df)

In [ ]:
# Construct the street network
G = street_network(nodes_df, arcs_df, weight = 'trip_time')

In [ ]:
# Select all the routes with more than 1 trip
opt_paths2 = []
for path in opt_paths:
    if len(path) > 1:
        opt_paths2.append(path)

In [ ]:
plot_taxi_route(G, opt_paths2[:5], nodes_df,'Optimal Sample Taxi Routes')

Retrieve the original paths of the taxis

In [ ]:
og_paths = get_og_path(trips_df)

In [ ]:
# Select all the routes with more than 1 trip
og_paths2 = []
for path in og_paths:
    if len(path) > 1:
        og_paths2.append(path)

In [ ]:
plot_taxi_route(G, og_paths2[:5], nodes_df,'Original Sample Taxi Routes')

Compare statistics of the taxi routings

In [ ]:
opt_stats = get_taxi_stats(opt_paths, trips_df)
og_stats = get_taxi_stats(og_paths, trips_df)

Summary statistics for the original taxi routing

In [ ]:
agg_stats(og_stats)

Summary statistics for the optimal taxi routing

In [ ]:
agg_stats(opt_stats)

In [ ]:
plot_stats(og_stats, opt_stats)

**Q:** What conclusions can you draw from the histograms above in terms of the difference between the original and the optimal taxi routings?

**A:** <font color='blue'> Total trip time and empty trip time are greatly reduced. However, to offset the reductions in trip time, the number of trips and on-trip percentage are increased.</font>

Another way to compare different taxi routings is to compare their distribution of taxis over time.

In [ ]:
day_data = get_day_dist(og_paths, opt_paths, times_df)

In [ ]:
plot_taxi_dist(day_data)

**Q:** What are the estimated original and optimal number of circulating taxis on this example day shown above?

**A:** <font color='blue'> Original number of circulating taxis: 7798; optimal number of circulating taxis: 1305.  </font>